In [88]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [95]:
from sophius.templates import *
from sophius.encode import Encoder
from sophius.modelgen import ConvModelGenerator

# Example usage
templates = [
    LinearTmpl(),
    BatchNorm2dTmpl(),
    ReLUTmpl(),
    LeakyReLUTmpl(),
    PReLUTmpl(),
    DropoutTmpl(),
    Dropout2dTmpl(),
    FlattenTmpl(),
    Conv2dTmpl(),
    MaxPool2dTmpl(),
    AvgPool2dTmpl(),
    GlobalAvgPool2dTmpl(),
]

encoder = Encoder()

for t in templates:
    t.gen_rand_config()
    encoded_layer = encoder.encode_template(t)
    
    print(t.type, t.get_learnable_params())
    # print(len(encoded_layer))
    print(encoded_layer)
    print()

LinearTmpl {'out_features': 2048}
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]

BatchNorm2dTmpl {}
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

ReLUTmpl {}
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

LeakyReLUTmpl {'negative_slope': 0.01}
[0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

PReLUTmpl {'all_channels': True}
[0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

DropoutTmpl {'p': 0.39999999999999997}
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]

Dropout2dTmpl {'p': 0.95}
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]

FlattenTmpl {}
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

Conv2dTmpl {'out_channels': 96, 'kernel_size': (1, 1), 'stride': (1, 1), 'padding': False}
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1]

MaxPool2dTmpl {'kernel_size': (2, 2), 'stride': (2, 2), 'padding': True}
[0 0 0 0 0 0 0 0

In [97]:
model_gen = ConvModelGenerator((3, 32, 32), 10, conv_num=6, lin_num=3)
model_tmpl = model_gen.generate_model_tmpl()

print(model_tmpl)

for t in model_tmpl.get_templates():
    print(t.type, t.get_learnable_params())

print()

for t in model_tmpl.get_templates()[:-1]:
    print(encoder.encode_template(t))

Conv2d       (128, 7, 7)    (6, 6)   (4, 4)  
PReLU        (128, 7, 7)   
MaxPool2d    (128, 4, 4)    (2, 2)   (2, 2)  
Conv2d       (96, 1, 1)     (3, 3)   (2, 2)  
BatchNorm2d  (96, 1, 1)    
ReLU         (96, 1, 1)    
MaxPool2d    (96, 1, 1)     (3, 3)   (3, 3)  
Flatten      96            
Linear       10            

Conv2dTmpl {'out_channels': 128, 'kernel_size': (6, 6), 'stride': (4, 4), 'padding': False}
PReLUTmpl {'all_channels': False}
MaxPool2dTmpl {'kernel_size': (2, 2), 'stride': (2, 2), 'padding': True}
Conv2dTmpl {'out_channels': 96, 'kernel_size': (3, 3), 'stride': (2, 2), 'padding': False}
BatchNorm2dTmpl {}
ReLUTmpl {}
MaxPool2dTmpl {'kernel_size': (3, 3), 'stride': (3, 3), 'padding': True}
FlattenTmpl {}
LinearTmpl {'out_features': 10}

[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1]
[0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 

In [98]:
for i in range(1000):
    m = model_gen.generate_model_tmpl()
    encoder.encode_model(m)

In [99]:
model_tmpl = model_gen.generate_model_tmpl()
vec = encoder.encode_model(model_tmpl)
print(vec)

[[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 